Calculating the hash directly is not working the way I want. 

Hence, I need to get the best image for each of the cholecSeg8k images. 

In [1]:
import sys
sys.path.append('../..')


In [2]:
from wand.image import Image
from os.path import join
import os
from utils.save_files import save_to_json

In [7]:
seq_ids = ['12', '01', '18', '25', '26', '27', '35', '43', '48','52']


In [8]:
dataset_train_dir = 'C:/Users/tal22/Documents/repositories/generate_binary_and_instance_masks_for_cholecseg8k/datasets/instance_cholec_v6_during_quality_control/train'

In [9]:
def get_file_id(filename):
    return int(filename.split('.')[0].split('_')[-1])
    
def closest_integers(n):
    # Generate the three closest integers on either side of n
    integers = [int(n) + i for i in range(-1, 2)]
    
    # Filter out any integers that are less than zero
    positive_integers = [i for i in integers if i >= 0]
    
    return positive_integers    

In [10]:
for seq_id in seq_ids:
    cholecseg8k_name = f'VID{seq_id}_seg8k'
    t50_name = f'VID{seq_id}_t50_full'  
    
    cholecseg8k_dir_path = join(dataset_train_dir, cholecseg8k_name, 'img_dir') 
    t50_dir_path = join(dataset_train_dir, t50_name, 'img_dir') 
    
    files_cholecseg8k_dir = os.listdir(cholecseg8k_dir_path)
    files_t50_dir = os.listdir(t50_dir_path)

    most_similar_dict =  {key: {'similar_file': '',
                                'metric_value': 1000} for key in files_cholecseg8k_dir}
    for seg8k_img in files_cholecseg8k_dir:
        print(f'{seg8k_img}........... ')
        seg8k_img_path = join(cholecseg8k_dir_path, seg8k_img)  
        seg8k_file_id = get_file_id(seg8k_img) 
        corresponding_t50_id = seg8k_file_id//25
        corresponding_t50_id_and_neighbors = closest_integers(corresponding_t50_id)
        
        # print(seg8k_img)
        # print(seg8k_file_id)
        # print(corresponding_t50_id_and_neighbors)
        for t50_id in corresponding_t50_id_and_neighbors:
            t50_img = files_t50_dir[t50_id]
            t50_img_path = join(t50_dir_path, t50_img)
            
            with Image(filename=seg8k_img_path) as base:
                with Image(filename=t50_img_path) as img:
                    base.fuzz = base.quantum_range * 0.20  # Threshold of 20%
                    _, result_metric = base.compare(img, metric='perceptual_hash')
                    # print(result_metric)
                    if result_metric < most_similar_dict[seg8k_img]['metric_value']:
                        most_similar_dict[seg8k_img]['similar_file'] = t50_img
                        most_similar_dict[seg8k_img]['metric_value'] = result_metric
        
    save_to_json(data=most_similar_dict,json_file_path=f'../../logs/similar_files/similar_files_VID{seq_id}') 
    
    

seg8k_video12_015750.png........... 
seg8k_video12_015751.png........... 
seg8k_video12_015752.png........... 
seg8k_video12_015753.png........... 
seg8k_video12_015754.png........... 
seg8k_video12_015755.png........... 
seg8k_video12_015756.png........... 
seg8k_video12_015757.png........... 
seg8k_video12_015758.png........... 
seg8k_video12_015759.png........... 
seg8k_video12_015760.png........... 
seg8k_video12_015761.png........... 
seg8k_video12_015762.png........... 
seg8k_video12_015763.png........... 
seg8k_video12_015764.png........... 
seg8k_video12_015765.png........... 
seg8k_video12_015766.png........... 
seg8k_video12_015767.png........... 
seg8k_video12_015768.png........... 
seg8k_video12_015769.png........... 
seg8k_video12_015770.png........... 
seg8k_video12_015771.png........... 
seg8k_video12_015772.png........... 
seg8k_video12_015773.png........... 
seg8k_video12_015774.png........... 
seg8k_video12_015775.png........... 
seg8k_video12_015776.png........... 
s

I need to go through all the values and choose the ones that are most similar. 

## Get best solutions

In [2]:
import sys
sys.path.append('../..')


In [8]:
from utils.read_files import read_from_json 
from utils.save_files import save_to_json
import os
from os.path import join

In [20]:
def refine_the_comparison_between_cholec_seg8k_and_T50(comparison_log_dir):
    
    total = 0
    for json_file in  os.listdir(comparison_log_dir):
        json_file_path = join(comparison_log_dir, json_file)
        seq_name = json_file.split('.')[0].split('_')[-1]
        data = read_from_json(json_file_path)    
        filtered = {}
        for key, value in data.items():
            similar_file = value['similar_file']
            metric_value = value['metric_value']
            
            if similar_file not in filtered:
                filtered[similar_file] = {
                    'keys': [],
                    'metric_values': [],
                    'min_metric_value': float('inf'),
                    'key_with_min_metric': None
                }
            
            grouped_entry = filtered[similar_file]
            grouped_entry['keys'].append(key)
            grouped_entry['metric_values'].append(metric_value)
            
            # Check for minimum metric value
            if metric_value < grouped_entry['min_metric_value']:
                grouped_entry['min_metric_value'] = metric_value
                grouped_entry['key_with_min_metric'] = key
                
        amount_of_filtered_share_files =  len(filtered.keys()) 
        total +=  amount_of_filtered_share_files    
        print(f'{seq_name} - {amount_of_filtered_share_files}' )
        save_to_json(data=filtered,json_file_path=f'../../logs/similar_files/filtered_comparison_with_large_minimums/filtered_comparison_{seq_name}.json') 
    
    print(f'Total amount of shared files - {total}')
            
     

In [21]:
refine_the_comparison_between_cholec_seg8k_and_T50(f'../../logs/similar_files/comparing_seg8k_T50')


VID01 - 58
VID12 - 30
VID18 - 9
VID25 - 14
VID26 - 17
VID27 - 16
VID35 - 13
VID43 - 23
VID48 - 12
VID52 - 32
Total amount of shared files - 224


In [24]:
def remove_the_values_with_high_minimums_final_filtering(comparison_log_dir, threshold=1):
    total = 0
    for json_file in  os.listdir(comparison_log_dir):
        json_file_path = join(comparison_log_dir, json_file)
        seq_name = json_file.split('.')[0].split('_')[-1]
        data = read_from_json(json_file_path)    
        filtered = {}
        for key, value in data.items():
            if value['min_metric_value'] > threshold:
                continue
            
            filtered[key] = {
                'most_similar': value['key_with_min_metric'],
                'metric': value['min_metric_value']
            } 
                
        amount_of_filtered_share_files =  len(filtered.keys()) 
        total +=  amount_of_filtered_share_files    
        print(f'{seq_name} - {amount_of_filtered_share_files}' )
        save_to_json(data=filtered,json_file_path=f'../../logs/similar_files/final_filtering/final_filtered_comparison_{seq_name}.json') 
    
    print(f'Total amount of shared files - {total}')
    

In [25]:
remove_the_values_with_high_minimums_final_filtering(f'../../logs/similar_files/filtered_comparison_with_large_minimums')


VID01 - 50
VID12 - 25
VID18 - 0
VID25 - 0
VID26 - 5
VID27 - 1
VID35 - 10
VID43 - 1
VID48 - 1
VID52 - 2
Total amount of shared files - 95


In [23]:

a

{'c': 4}

In [27]:
len(grouped.keys())

58

Report these results.